In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import logging
import pickle

In [2]:
with open('pkl/data_vecor.pkl','rb') as file:
    data_a=pickle.load(file)

In [3]:
data_a.head()

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,keyword_vector_54,keyword_vector_55,keyword_vector_56,keyword_vector_57,keyword_vector_58,keyword_vector_59,keyword_vector_60,keyword_vector_61,keyword_vector_62,keyword_vector_63
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,...,0.182745,0.531644,-0.277505,-0.337594,-0.470887,-0.333244,-0.105863,0.155339,-0.305796,0.068118
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,...,0.182745,0.531644,-0.277505,-0.337594,-0.470887,-0.333244,-0.105863,0.155339,-0.305796,0.068118
2,3867,2,3,4,0.0,32.0,13.0,9.531250,305.0,1.0,...,0.182745,0.531644,-0.277505,-0.337594,-0.470887,-0.333244,-0.105863,0.155339,-0.305796,0.068118
3,3861,1,2,20,0.0,3.0,234.0,222.333328,667.0,1.0,...,0.182745,0.531644,-0.277505,-0.337594,-0.470887,-0.333244,-0.105863,0.155339,-0.305796,0.068118
4,3862,5,1,8,1.0,NaN,NaN,NaN,NaN,NaN,...,0.182745,0.531644,-0.277505,-0.337594,-0.470887,-0.333244,-0.105863,0.155339,-0.305796,0.068118


In [4]:
feature_cols = [x for x in data_a.columns if x not in ('label', 'uid', 'qid', 'dt', 'day')]
# target编码

In [5]:
# train_label = train[(train['day'] > train_label_feature_end)]
# print(len(train_label))
train_label=2593669

In [6]:
X_train_all = data_a.iloc[:train_label][feature_cols]
y_train_all = data_a.iloc[:train_label]['label']
test = data_a.iloc[train_label:]

logging.info("train shape %s, test shape %s", X_train_all.shape, test.shape)

In [7]:
print(X_train_all.shape)
print(test.shape)

(2593669, 638)
(1141683, 642)


In [8]:
model_lgb = LGBMClassifier(boosting_type='gbdt', num_leaves=64, learning_rate=0.01, n_estimators=5000,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=1000, n_jobs=-1, silent=True)

In [ ]:
fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                     y_train_all.iloc[train_idx], \
                                     y_train_all.iloc[val_idx]
    model_lgb.fit(X_train, y_train,
                  eval_metric=['logloss', 'auc'],
                  eval_set=[(X_val, y_val)],
                  early_stopping_rounds=10)

[1]	valid_0's auc: 0.776631	valid_0's binary_logloss: 0.438116
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.780602	valid_0's binary_logloss: 0.436592
[3]	valid_0's auc: 0.781783	valid_0's binary_logloss: 0.43511
[4]	valid_0's auc: 0.786744	valid_0's binary_logloss: 0.433681
[5]	valid_0's auc: 0.787681	valid_0's binary_logloss: 0.432279
[6]	valid_0's auc: 0.787936	valid_0's binary_logloss: 0.430911
[7]	valid_0's auc: 0.788348	valid_0's binary_logloss: 0.429583
[8]	valid_0's auc: 0.788892	valid_0's binary_logloss: 0.428277
[9]	valid_0's auc: 0.788918	valid_0's binary_logloss: 0.427013
[10]	valid_0's auc: 0.790111	valid_0's binary_logloss: 0.425776
[11]	valid_0's auc: 0.790373	valid_0's binary_logloss: 0.424583
[12]	valid_0's auc: 0.790516	valid_0's binary_logloss: 0.423409
[13]	valid_0's auc: 0.790316	valid_0's binary_logloss: 0.422268
[14]	valid_0's auc: 0.790673	valid_0's binary_logloss: 0.421149
[15]	valid_0's auc: 0.791	valid_0's binary_logloss: 0

In [ ]:
sub = pd.read_csv(f'/home/huangqiao/DataMiningCode/project-expert/data/test.txt', sep='\t', header=None)
sub.columns = ['qid', 'uid', 'dt']
sub['label'] = model_lgb.predict_proba(test[feature_cols])[:, 1]
sub.to_csv('result-hebing-5000.txt', index=None, header=None, sep='\t')